<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" width="140px" alt="EPFL_logo">

## Image Processing Laboratory Notebooks
---

This Jupyter Notebook is part of a series of computer laboratories designed
to teach image-processing programming; they are running on EPFL's Noto server. They are the practical complement of the theoretical lectures of the EPFL's Master course 
[**MICRO-512 Image Processing II**](https://moodle.epfl.ch/course/view.php?id=522) taught by Prof. M. Unser and Prof. D. Van de Ville.

The project is funded by the Center for Digital Education and the School of Engineering. It is owned by the [Biomedical Imaging Group](http://bigwww.epfl.ch/). 
The distribution or reproduction of the notebook is strictly prohibited without the written consent of the authors.  &copy; EPFL 2025.

**Authors**: 
    Sébastien Herbreteau, 
    [Daniel Sage](mailto:daniel.sage@epfl.ch), and
    [Sepand Kashani](mailto:sepand.kashani@epfl.ch),
    
---
# Lab 7.1: Deconvolution
**Released**: Thursday, May 15, 2025

**Submission deadline**: Monday, May 26, 2025, before 23:59 on [Moodle](https://moodle.epfl.ch/course/view.php?id=463)

**Grade weight**: Lab 7 (21 points), 7.5 % of the overall grade

**Related lectures**: Chapter 9 and 10

Double-click on this cell, fill your name and SCIPER number below to verify your identity in Noto and set the seed for random results.
:::{attention} Please write down your name and SCIPER! 
### Student Name: 
### SCIPER:
:::

In [ ]:
import getpass
# This line recovers your camipro number to mark the images with your ID
uid = int(getpass.getuser().split('-')[2]) if len(getpass.getuser().split('-')) > 2 else ord(getpass.getuser()[0])
print(f'SCIPER: {uid}')

## Imports
In the next cell we import Python libraries we will use throughout the lab.

In [ ]:
# Import standard required packages for this exercise
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import skimage
from interactive_kit import imviewer as viewer

# Configure plotting as dynamic
%matplotlib widget

# Deconvolution (14 points)

In this lab, you will study deconvolution algorithms. You will implement simple deconvolution approaches yourself and explore more regularization techniques with Pyxu in the second lab.

## 1. Inverse filtering

In order to preserve the identity of a (famous) person in an image $s$, a naive software engineer at a Big Tech company decides to blur the entire image $s$ with a centered Gaussian kernel, denoted $h$. In other words, the corruption model he uses is simply: $$y = h * s\,,$$
where $s$ is the true signal, $h$ is the blurring kernel (with **periodic padding**) and $y$ the corrupted image. The blurred image $y$ is available at `images/blurred.tif`.

Before revealing the identity of the person in `blurred.tif` in part [1.B](#1.B.-Revealing-the-identity-of-the-blurred-celebrity-(2-points)), you are going to experiment first with the test image `einstein.tif`.

### 1.A. Reproducing blur on a test image (2 points)

For **1 point**, complete the function `filtering_periodic` that implements the operation $h * s$ with periodic padding **using 2D discrete Fourier transform (DFT)** and test it with the test image of Albert Einstein (`images/einstein.tif`) and the given centered Gaussian kernel $h$ with standard deviation $\sigma_h=20$. You should use the NumPy functions `np.fft.fft2` and `np.fft.ifft2`.

The function `filtering_periodic` takes as inputs
 * `s`: the input image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `y`, the filtered image of the same size as `s`.

*Hint:* You may want to use the given function `kernel_to_image` below.

**Beware:** `filtering_periodic` must output an image with pixels in $\mathbb{R}$ (not in $\mathbb{C}$).

In [ ]:
def get_2D_Gaussian_kernel(sigma_h):
    """ Returns a centered Gaussian kernel h of size (2k+1)*(2k+1) """
    k = int(np.ceil(3*sigma_h))
    kernel_h_1D = np.exp(-np.arange(-k, k+1)**2 / (2 * sigma_h**2))
    kernel_h_1D /= kernel_h_1D.sum() # normalization so that the sum of the coefficients equals 1
    kernel_h_2D = kernel_h_1D.reshape(-1, 1) @ kernel_h_1D.reshape(1, -1)
    return kernel_h_2D
    
def kernel_to_image(h, s):
    """ Converts the kernel h of size (2k+1)x(2k+1) to an image of the same size as s, denoted h_ext, 
    so that the relation h * s = y <=> DFT(h_ext) .* DFT(s) = DFT(y) holds """
    h_ext = np.zeros(s.shape)
    k = h.shape[0] // 2
    h_ext[:(2*k+1), :(2*k+1)] = h
    return np.roll(h_ext, (-k, -k), axis=(0, 1))
    
def filtering_periodic(s, h):
    """ Returns h * s """
    # YOUR CODE HERE
    return y

In [ ]:
# Import Einstein image
einstein = skimage.io.imread('images/einstein.tif').astype(np.float64)

# Display result
sigma_h = 20
kernel_h_2D = get_2D_Gaussian_kernel(sigma_h)
blurred_einstein = filtering_periodic(einstein, kernel_h_2D)
plt.close('all')
view = viewer([einstein, blurred_einstein, kernel_h_2D], widgets=True, hist=False, axis=True, cmap='gray')

Although it may seem that the person has been anonymized, this way of corrupting data is not safe as it is completely reversible as long as the Gaussian kernel $h$ is known. For **1 point**, implement the function `inverse_filtering_periodic` **using 2D discrete Fourier transform (DFT)** that recovers the true signal $s$ from its corrupted version $y$.


The function `inverse_filtering_periodic` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `s`, the original image of the same size as `y`.

In [ ]:
def inverse_filtering_periodic(y, h):
    """ Returns s such that y = h * s """
    # YOUR CODE HERE
    return s

deblurred_einstein = inverse_filtering_periodic(blurred_einstein, kernel_h_2D)
absolute_error = np.abs(deblurred_einstein - einstein) # due to numerical errors
plt.close('all')
view = viewer([deblurred_einstein, absolute_error], widgets=True, hist=False, axis=True, cmap='gray')

In [ ]:
# Perform sanity check on a random image
s_random = np.random.rand(40, 40)
h_random = np.random.rand(5, 5)
if np.allclose(inverse_filtering_periodic(filtering_periodic(s_random, h_random), h_random), s_random) \
    and np.max(np.abs(np.imag(filtering_periodic(s_random, h_random)))) == 0 \
    and np.max(np.abs(np.imag(inverse_filtering_periodic(s_random, h_random)))) == 0:
    print('Congratulations! Your code seems to be working.') 
else:
    print('WARNING!\nYour code is not correct!') 

### 1.B. Revealing the identity of the blurred celebrity (2 points)

For **2 points**, using the function `inverse_filtering_periodic`, reveal the identity of the blurred person at `images/blurred.tif`, given that the standard deviation $\sigma_h$ of the Gaussian kernel used for corruption is **an integer**. 

*Hint:* supply a list of potential $\sigma_h$ (stored in `list_sigmas`), run the deconvolution for each $\sigma_h$ and store the images in `list_deblurred_images`.

In [ ]:
blurred_person = skimage.io.imread('images/blurred.tif').astype(np.float64)

list_sigmas = [0]
list_deblurred_images = [blurred_person]

# YOUR CODE HERE

plt.close('all')
view = viewer(list_deblurred_images, title=[str(elt) for elt in list_sigmas], widgets=True, hist=False, axis=True, cmap='gray')

In the next cell, assign to the variable `sigma_blur` the standard deviation used for blurring the hidden person.

In [ ]:
# Standard deviation used for blurring the hidden celebrity
sigma_blur = None
# YOUR CODE HERE

In [ ]:
# Perform sanity check on sigma_blur
if not (0 < sigma_blur < 50 or np.round(sigma_blur) != sigma_blur): 
    print('WARNING!\nThe selected standard deviation is not really reasonable.')

### 1.C. Robustness to mirror padding (1 point)

For **1 point**, **using the function** `filtering_periodic`, implement the function `filtering_mirror` that filters the input image with a mirror padding (a.k.a. reflect padding) and observe that the function `inverse_filtering_periodic` defined above does not work for deblurring with this specific padding. 

The function `filtering_mirror` takes as inputs
 * `s`: the input image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 
and returns `y`, the filtered image with mirror padding of the same size as `s`.

$\Rightarrow$ Inverse filtering is not robust to a slight change in the padding mode!


*Hint:* You may want to use the function `np.pad` with reflect mode.

In [ ]:
def filtering_mirror(s, h):
    """ Returns h * s with mirror padding """ 
    # YOUR CODE HERE
    return y

In [ ]:
sigma_h = 20
kernel_h_2D = get_2D_Gaussian_kernel(sigma_h)
blurred_einstein_mirror = filtering_mirror(einstein, kernel_h_2D)
deblurred_einstein_mirror = inverse_filtering_periodic(blurred_einstein_mirror, kernel_h_2D)
plt.close('all')
view = viewer([deblurred_einstein_mirror, blurred_einstein_mirror], widgets=True, hist=False, axis=True, cmap='gray')

## 2. Regularization

In order to make it more difficult for potential attackers, the engineer decides to add white Gaussian noise $n \sim \mathcal{N}(0, \sigma_n^2 I)$ to the corruption model (still periodic padding):
$$y = h * s + n\,,$$
with $\sigma_n = 1$ (almost imperceptible noise). 

Observe that the function `inverse_filtering_periodic` defined above is not enough to recover the true signal $s$.

$\Rightarrow$ Inverse filtering is not robust to the addition of a weak noise!

In [ ]:
np.random.seed(1234) # for reproductibility, do not modify
sigma_h = 20
kernel_h_2D = get_2D_Gaussian_kernel(sigma_h)
blurred_einstein = filtering_periodic(einstein, kernel_h_2D) 
blurred_and_weakly_noisy_einstein = blurred_einstein + 1.0 * np.random.randn(*einstein.shape)
recovered_einstein = inverse_filtering_periodic(blurred_and_weakly_noisy_einstein, kernel_h_2D)
plt.close('all')
view = viewer([blurred_einstein, blurred_and_weakly_noisy_einstein, recovered_einstein], widgets=True, hist=False, axis=True, cmap='gray')

To tackle this problem, the least-squares method consists in solving:
$$s^\ast = \arg \min_s J_{\text{LS}}(s, y)\,,$$
where $J_{\text{LS}}(s, y) = \|h * s - y\|_2^2$. Note that, in this case, the least-squares solution coincides with the maximum-likelihood (ML) estimation solution. However, from a global perspective, ML deconvolution is ill-posed. It is better to constrain the solution by imposing explicit regularization constraints:
$$s^\ast_\lambda = \arg \min_s J_{\text{LS}}(s,y) + \lambda J_\text{reg}(s)\,,$$
where $\lambda \geq 0$ is an hyperparameter.

### 2.A. Tikhonov regularization (5 points)

In one of its simplest form, Tikhonov regularization is obtained with $J_\text{reg}(s) = \|Ls\|_2^2$ where $L$ is the Laplacian operator that penalizes oscillations.  For **2 points**, complete the function `solve_tikhonov` that computes the **formal solution** $s^\ast_\lambda$ using **discrete Fourier transform (DFT)**.


The function `solve_tikhonov` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 * `lam` (scalar): the hyperparameter $\lambda \geq 0$, 
 
and returns `solution`, the formal solution $s^\ast_\lambda$.


*Hints:*
    
1. The gradient of $J_{\text{LS}}(s,y) + \lambda J_\text{reg}(s)$ with regard to $s$ is equal to $2( (H^\top H + \lambda L^\top L) s -  H^\top y)$ where $H$ is the matrix formulation of convolution with kernel $h$ (in the discretized forward model). **Set it to zero to get the linear system for which $s^\ast_\lambda$ is solution**, then go to Fourier.
    
2. For the sake of simplicity, we use the $3 \times 3$ Laplacian operator, defined as $Ls = l * s$ with $$l = \begin{pmatrix} 0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0 \\
\end{pmatrix}\,.$$
    
3. Since all convolutions have symmetric kernels, $H^\top = H$ and $L^\top = L$.

In [ ]:
def solve_tikhonov(y, h, lam):
    r""" Returns the formal solution s_\lambda^\ast """
    # YOUR CODE HERE
    return solution

In [ ]:
# Perform sanity check for solve_tikhonov
np.random.seed(5678) # for reproductibility, do not modify
y_sanity_check = np.random.rand(16, 16)
h_sanity_check = np.array([[0.05, 0.1, 0.05], [0.1, 0.4, 0.1], [0.05, 0.1, 0.05]])
lam_sanity_check = 1e-1
solution_sanity_check = np.array([[0.43401456, 0.37066355, 0.47667136, 0.5514082 ],
       [0.56186321, 0.42420904, 0.5007256 , 0.49477319],
       [0.62218151, 0.43939143, 0.53553075, 0.53750101],
       [0.58575311, 0.47531916, 0.54754149, 0.59132934]])
if np.allclose(solution_sanity_check, solve_tikhonov(y_sanity_check, h_sanity_check, lam_sanity_check)[:4, :4]):
    print('Congratulations! Your function "solve_tikhonov" seems to be working.') 
else:
    print('WARNING!\nYour function "solve_tikhonov" is not correct!')

Observe the effect of the regularization hyperparameter $\lambda$ for deblurring the Einstein image $y$.

*Reminder:* `Extra Widgets` needs to be clicked to see the effect of deconvolution with `IntSlider`.

In [ ]:
# Declare slider
reg_slider = widgets.IntSlider(value=-8, min=-8, max=8, step=1, description='lam = 10^x')
activation_button = widgets.Button(description='Regularize')

# Declare callback
def callback(img):
    reg = reg_slider.value
    output = solve_tikhonov(img, kernel_h_2D, 10**reg)
    return output

plt.close('all')
view = viewer(blurred_and_weakly_noisy_einstein, new_widgets=[reg_slider, activation_button], callbacks=[callback], widgets=True)

The **signal to noise ratio** (SNR), expressed in decibels (dB), is defined between a ground truth image $s$ and its recovered version $\hat{s}$ as follows:
    $$\operatorname{SNR}(s,\hat{s}) = 10 \log_{10} \left(\frac{\sum_{i} s_i^2}{\sum_{i} (s_i - \hat{s}_i)^2}\right)\,.$$
Complete the function `SNR` below that implements it. The function `SNR` takes as inputs

 * `s`: the ground truth image, 
 * `s_hat`: the estimated image, 
 
and returns `snr` (float), the signal to noise ratio.

In [ ]:
def SNR(s, s_hat):
    # YOUR CODE HERE
    return snr

In [ ]:
# Sanity check for the function SNR
s1 = np.linspace(0.8, 1, 10).reshape(-1, 1) @ np.linspace(0.9, 1, 10).reshape(1, -1)
s2 = np.linspace(0.85, 1, 10).reshape(-1, 1) @ np.linspace(0.99, 1, 10).reshape(1, -1)
if np.allclose(SNR(s1, s2), 21.58120) and np.allclose(SNR(s2, s1), 22.20615):
    print('Congratulations! Your function "SNR" seems to be working.') 
else:
    print('WARNING!\nYour function "SNR" is not correct!')

For **1 point** answer the following MCQ:

* Among the following values of $\lambda$, which one maximizes the SNR between $s$ and $s^\ast_\lambda$ ?
1. $\lambda = 10^{-6}$
2. $\lambda = 10^{-4}$
3. $\lambda = 10^{-2}$
4. $\lambda = 10^{0}$
5. $\lambda = 10^{2}$
6. $\lambda = 10^{4}$
7. $\lambda = 10^{6}$

Use the next cell to experiment or confirm your hypothesis and then modify the variable `answer` in the cell right after it to reflect your choices. 

In [ ]:
# YOUR CODE HERE

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

#### Behavior at the limits

**For 1 point**, answer the following MCQ:
* What is the SNR of the least-squares solution, i.e. when $\lambda=0$ (rounded to the nearest integer) ? (**1 point**)
1. $-514$ dB
2. $-414$ dB
3. $-314$ dB
4. $-214$ dB
5. $-114$ dB
6. $0$ dB
7. $14$ dB

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

**For 1 point** answer the following MCQ:
* What is $s_\lambda^\ast$ when $\lambda$ tends to infinity ? 

**Note:** $\mathbf{0}$ and $\mathbf{1}$ denote the all-zeros vector and the all-ones vector of the same size as $y$, respectively.

1. $y$
2. $\mathbf{0}$
3. $\mathbf{1}$
4. $Hy$
5. $H \mathbf{1}$
6. $\operatorname{mean}(y) \mathbf{1}$
7. $\operatorname{median}(y) \mathbf{1}$

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)):
    print('WARNING!\nPossible answers are integers between 1 and 7.')

### 2.B. TV regularization (4 points)

Another very popular type of regularization is the Total variation regularization $J_{\text{TV}}(s) = \lambda \| \nabla s\|_1$, which favors piecewise-constant solutions. Complete the function `solve_tv` using the **Proximal Gradient Descent** algorithm  to solve:
    $$s^\ast_\lambda = \arg \min_s \frac{1}{2} \|h * s - y\|_2^2 + \lambda  \| \nabla s\|_1\,,$$
where $\lambda \geq 0$ is an hyperparameter.

The function `solve_tv` takes as inputs
 * `y`: the input corrupted image, 
 * `h`: the centered kernel of size $(2k+1)\times(2k+1)$,
 * `lam` (scalar): the hyperparameter $\lambda \geq 0$, 
 * `nb_iterations` (int): fixed to 100 (do not modify), 
 
and returns `solution`, the formal solution $s^\ast_\lambda$.

*Hint:* Use the function [`skimage.restoration.denoise_tv_chambolle`](https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_tv_chambolle) with the correct `weight` parameter for computing the proximal step, as there exists no closed-form expression.
    
**Note:** Since [`skimage.restoration.denoise_tv_chambolle`](https://scikit-image.org/docs/stable/api/skimage.restoration.html#skimage.restoration.denoise_tv_chambolle) is also an iterative function, the algorithm may take some time.

In [ ]:
def solve_tv(y, h, lam, nb_iterations=100):
    s = y # initialization
    L = 1.0 # Lipschitz constant of data term
    gamma = 1/L
    for _ in range(nb_iterations):
        # YOUR CODE HERE
    solution = s
    return solution

In [ ]:
# Perform sanity check for solve_tv
np.random.seed(5678) # for reproductibility, do not modify
y_sanity_check = np.random.rand(16, 16)
h_sanity_check = np.array([[0.05, 0.1, 0.05], [0.1, 0.4, 0.1], [0.05, 0.1, 0.05]])
lam_sanity_check = 5e-2
solution_sanity_check = np.array([[0.37104013, 0.21921995, 0.39198927, 0.49997054],
       [0.5447789 , 0.48547633, 0.51157949, 0.50653231],
       [0.57900974, 0.46908567, 0.51937762, 0.52004448],
       [0.66397081, 0.46555742, 0.52212   , 0.57628085]])
if np.allclose(solution_sanity_check, solve_tv(y_sanity_check, h_sanity_check, lam_sanity_check)[:4, :4]):
    print('Congratulations! Your function "solve_tv" seems to be working.') 
else:
    print('WARNING!\nYour function "solve_tv" is not correct!')

Observe the effect of the regularization with hyperparameter $\lambda = 10^{-1}$ for deblurring the Einstein image $y$.

**Note**: `solve_tv` takes about 20 seconds for 100 iterations.

In [ ]:
lam = 1e-1
recovered_einstein = solve_tv(blurred_and_weakly_noisy_einstein, kernel_h_2D, lam, nb_iterations=100)

In [ ]:
plt.close('all')
view = viewer([recovered_einstein, blurred_and_weakly_noisy_einstein], widgets=True, hist=False, axis=True, cmap='gray')

#### Behavior at the limits

Answer the following MCQ:
* What is $s_\lambda^\ast$ when $\lambda$ tends to infinity ? 

**Note:** $\mathbf{0}$ and $\mathbf{1}$ denote the all-zeros vector and the all-ones vector of the same size as $y$, respectively.

1. $y$
2. $\mathbf{0}$
3. $\mathbf{1}$
4. $Hy$
5. $H \mathbf{1}$
6. $\operatorname{mean}(y) \mathbf{1}$
7. $\operatorname{median}(y) \mathbf{1}$

Modify the variable `answer` in the next cell to reflect your choices. 

In [ ]:
# Assign your answer to this variable
answer = None
# YOUR CODE HERE

In [ ]:
if not answer in list(range(1, 8)): 
    print('WARNING!\nPossible answers are integers between 1 and 7.')

Solving inverse problems by adding a term of regularization is powerful but requires implementing a specific optimization algorithm for each form of the regularizer and the forward operator. In the second part of this lab, we are going to see that the library Pyxu, developed at EPFL, may be helpful for this. 

🎉 Congratulations on finishing the first part of the Inverse Problem lab!! 🎉

Make sure to save your notebook (you might want to keep a copy on your personal computer) and upload it to Moodle, **in a zip file with the other notebook of this lab.**

* Keep the name of the notebook as: *1_deconvolution.ipynb*,
* Name the zip file: *inverse_problem_lab.zip*.